In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import math
import matplotlib.pyplot as plt
import time
from queue import Queue
import requests
import pygame
import threading
import random
import string

import firebase_admin
from firebase_admin import credentials, db
import time
from datetime import datetime

# 미디어파이프 모듈 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

font = cv2.FONT_HERSHEY_DUPLEX
count_position = (20, 30)

# 각 관절의 인덱스
JOINTS = {
    "right_shoulder": 12,
    "right_elbow": 14,
    "right_wrist": 16,
    "left_shoulder": 11,
    "left_elbow": 13,
    "left_wrist": 15,
    "right_hip": 24,
    "right_knee": 26,
    "right_ankle": 28,
    "left_hip": 23,
    "left_knee": 25,
    "left_ankle": 27,
}

# Google Cloud 프로젝트 및 서비스 계정 키
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/altong/Desktop/project/popoteststt-31bc6d0f3189.json"

# typecast
actor_id = "6359e7ea258d1b6dc3abe6e6"
emotion_tone_preset = "normal-1"
typecast_API_KEY = '__pltPCLsQuV6MaS4StLkxwkChB2dAeGMwrRvcrY2AvwE'

def current_time():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Initialize Firebase
cred = credentials.Certificate("altproject-57038-firebase-adminsdk-smksf-df396f61fb.json")
firebase_admin.initialize_app(cred, {'databaseURL': 'https://altproject-57038-default-rtdb.firebaseio.com'})

# 데이터베이스의 'exercise_data' 노드에 대한 참조 생성
ref = db.reference("exercise_data")

def update_exercise_data(exercise_type, pullup_count=0, pushup_count=0, squat_count=0, path=""):
    current = time.strftime("KmpVKNlNscaxsC6wQfL2wyZg0EU2/%Y-%m-%d")

    # 기존 데이터 가져오기
    existing_data = ref.child(current).get()

    if existing_data:
        if exercise_type in existing_data:
            existing_data[exercise_type]["count"] += locals()[f"{exercise_type.lower()}_count"]
            existing_data[exercise_type]["weekday"] = time.localtime().tm_wday + 1
            pathList = existing_data[exercise_type].get("path", [])
            pathList.append(path)
            existing_data[exercise_type]["path"] = pathList
        else:
            # If key is missing, create it with initial values
            existing_data[exercise_type] = {
                "count": locals()[f"{exercise_type.lower()}_count"],
                "weekday": time.localtime().tm_wday + 1,
                "path": [path]
            }
        print(existing_data[exercise_type])

        # 데이터베이스 업데이트
        ref.child(current).update(existing_data)
    else:
        # 기존 데이터가 없으면 새로 생성
        existing_data = {
 
        }
        if exercise_type == "PullUp" :
            existing_data["PullUp"] = dict(
                    count= pullup_count,
                    weekday= time.localtime().tm_wday + 1,
                    path= [path]
                ),
            existing_data["PullUp"] = existing_data["PullUp"][0]
            print(existing_data["PullUp"])
        if exercise_type == "PushUp" :
            existing_data["PushUp"] = dict(
                    count= pushup_count,
                    weekday= time.localtime().tm_wday + 1,
                    path= [path]
                ),
            existing_data["PushUp"] = existing_data["PushUp"][0]
            print(existing_data["PushUp"])
        if exercise_type == "Squat" :
            existing_data["Squat"] = dict(
                    count= squat_count,
                    weekday= time.localtime().tm_wday + 1,
                    path= [path]
                ),
            existing_data["Squat"] = existing_data["Squat"][0]
            print(existing_data["Squat"])
 

        # 데이터베이스 업데이트
        ref.child(current).update(existing_data)



def play_audio(filename):
   
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()
    clock = pygame.time.Clock()
    while pygame.mixer.music.get_busy():
        clock.tick(30)  

# typecast
def text_to_speech(text, lang='ko', filename='outSound'):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {typecast_API_KEY}'
    }

    r = requests.get('https://typecast.ai/api/actor', headers=headers)
    my_actors = r.json()['result']
    my_first_actor = my_actors[0]
    my_first_actor_id = my_first_actor['actor_id']

    json_data = {
        'text': text,
        'lang': lang,
        'xapi_hd': True,
        'actor_id': my_first_actor_id,
        'model_version': 'latest',
        'emotion_tone_preset': emotion_tone_preset,
        # 'xapi_audio_format' : "mp3"

    }

    response = requests.post('https://typecast.ai/api/speak', headers=headers, json=json_data)

    speak_url = response.json()['result']['speak_v2_url']
 
    if response.status_code == 200:
        for _ in range(120):
        # Download the audio file
            audio_response = requests.get(speak_url,headers=headers)
            # print(audio_response)
            ret = audio_response.json()['result']
            print(ret)
            if ret['status'] == 'done':
                audio_response = requests.get(ret['audio_download_url'])
                # print(ra)
                with open(filename, 'wb') as f:
                    for chunk in audio_response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                break
            else:
                time.sleep(1)
 
       
       
        print("speaking start")

        pygame.mixer.init()
        play_audio(filename)

        pygame.mixer.music.stop()
        pygame.mixer.quit()
        print("speaking end")

        os.remove(filename)
        print(f"File '{filename}' deleted successfully.")
    else:
        print(f"Error: Typecast API request failed with status code {response.status_code}")



# 각 앵글을 계산하는 함수
def calculate_angle(a, b, c):
    radians = math.atan2(c[1] - b[1], c[0] - b[0]) - math.atan2(a[1] - b[1], a[0] - b[0])
    angle = math.degrees(radians)
    return angle + 360 if angle < 0 else angle


# Global Variables
exercise_queue = Queue()

user_state = False


def producer_thread():

    while True:
           # Firebase Realtime Database에서 데이터 가져오기
        ref = db.reference('current_routine')
        # Read data
        current_routine = ref.get()

        if current_routine:
            rest_time = current_routine.get('rest_time')
            routine_data = current_routine.get('routine', [])

            # Loop through exercises in the routine
            for exercise_data in routine_data:
                counts = exercise_data.get('count')
                exercise = exercise_data.get('exercise')
                set_count = exercise_data.get('set')

            # Exercise logic (replace this with your actual exercise execution logic)
                print(f"Exercise: {exercise}, Count: {counts}, Set: {set_count}")
                time.sleep(2)  # Example: wait for 2 seconds
                # 사용자에게 입력받기
                exercise_type = exercise
                input_count = counts
                input_rest = rest_time
                input_sets = set_count
                today = datetime.now().strftime("%Y-%m-%d")
                generate_random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
               
                if exercise :
                    user_state = True
                    if user_state:
                        for set_count in range(int(input_sets)):
                            print(f"\n{set_count + 1}세트를 시작합니다.\n")
                            text_to_speech(f'{exercise}{set_count + 1}세트를 시작할게요', lang='ko', filename='outSound.wav')
                            global exercise_queue

                            # 카메라 설정
                            cap = cv2.VideoCapture(0)

                            # 카운트 초기화
                            count = 0
                            count_s = -1
                            detected = False
                            less_count = False
                            more_count = False

                            # VideoWriter 설정
                            exercise_file_path = f'/home/altong/Desktop/project/test/public/{today}_{exercise}_record_{set_count+1}_{generate_random_string}.mp4'
                            fps = 5.40
                            fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 인코딩 포맷 문자
                            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                            size = (width, height)  # 프레임 크기
                            out = cv2.VideoWriter(exercise_file_path, fourcc, fps, size)
 # VideoWriter 객체 생성
   


                            # 운동 감지 및 동영상 녹화
                            start_time = time.time()
                            last_count_time = start_time

                            while cap.isOpened():
                                ret, frame = cap.read()
                                if not ret:
                                    break

                            # 좌우 반전
                                frame = cv2.flip(frame, 1)  # 1은 수평 방향으로 뒤집음

                                # RGB로 변환
                                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
   
                                # 미디어파이프를 사용하여 관절 포인트 감지
                                results = pose.process(rgb_frame)

                                # 현재 프레임의 시간 계산
                                current_time = time.time() - start_time

                                if results.pose_landmarks:
                                # 앵글 계산
                                    right_shoulder = (int(results.pose_landmarks.landmark[JOINTS["right_shoulder"]].x * frame.shape[1]),
                                  int(results.pose_landmarks.landmark[JOINTS["right_shoulder"]].y * frame.shape[0]))
                                    right_elbow = (int(results.pose_landmarks.landmark[JOINTS["right_elbow"]].x * frame.shape[1]),
                               int(results.pose_landmarks.landmark[JOINTS["right_elbow"]].y * frame.shape[0]))
                                    right_wrist = (int(results.pose_landmarks.landmark[JOINTS["right_wrist"]].x * frame.shape[1]),
                               int(results.pose_landmarks.landmark[JOINTS["right_wrist"]].y * frame.shape[0]))

                                    left_shoulder = (int(results.pose_landmarks.landmark[JOINTS["left_shoulder"]].x * frame.shape[1]),
                                 int(results.pose_landmarks.landmark[JOINTS["left_shoulder"]].y * frame.shape[0]))
                                    left_elbow = (int(results.pose_landmarks.landmark[JOINTS["left_elbow"]].x * frame.shape[1]),
                              int(results.pose_landmarks.landmark[JOINTS["left_elbow"]].y * frame.shape[0]))
                                    left_wrist = (int(results.pose_landmarks.landmark[JOINTS["left_wrist"]].x * frame.shape[1]),
                              int(results.pose_landmarks.landmark[JOINTS["left_wrist"]].y * frame.shape[0]))

                                    right_hip = (int(results.pose_landmarks.landmark[JOINTS["right_hip"]].x * frame.shape[1]),
                             int(results.pose_landmarks.landmark[JOINTS["right_hip"]].y * frame.shape[0]))
                                    right_knee = (int(results.pose_landmarks.landmark[JOINTS["right_knee"]].x * frame.shape[1]),
                               int(results.pose_landmarks.landmark[JOINTS["right_knee"]].y * frame.shape[0]))
                                    right_ankle = (int(results.pose_landmarks.landmark[JOINTS["right_ankle"]].x * frame.shape[1]),
                                int(results.pose_landmarks.landmark[JOINTS["right_ankle"]].y * frame.shape[0]))

                                    left_hip = (int(results.pose_landmarks.landmark[JOINTS["left_hip"]].x * frame.shape[1]),
                            int(results.pose_landmarks.landmark[JOINTS["left_hip"]].y * frame.shape[0]))
                                    left_knee = (int(results.pose_landmarks.landmark[JOINTS["left_knee"]].x * frame.shape[1]),
                              int(results.pose_landmarks.landmark[JOINTS["left_knee"]].y * frame.shape[0]))
                                    left_ankle = (int(results.pose_landmarks.landmark[JOINTS["left_ankle"]].x * frame.shape[1]),
                               int(results.pose_landmarks.landmark[JOINTS["left_ankle"]].y * frame.shape[0]))

                                    # 각 앵글 계산
                                    right_shoulder_elbow_wrist_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                                    left_shoulder_elbow_wrist_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                                    right_hip_knee_ankle_angle = calculate_angle(right_hip, right_knee, right_ankle)
                                    left_hip_knee_ankle_angle = calculate_angle(left_hip, left_knee, left_ankle)
                                    right_knee_hip_shoulder_angle = calculate_angle(right_knee, right_hip, right_shoulder)
                                    left_knee_hip_shoulder_angle = calculate_angle(left_knee, left_hip, left_shoulder)
                                    right_ankle_hip_shoulder_angle = calculate_angle(right_ankle, right_hip, right_shoulder)
                                    left_ankle_hip_shoulder_angle = calculate_angle(left_ankle, left_hip, left_shoulder)

                                    # 화면에 카운트 표시
                                    cv2.putText(frame, f"Count: {count}", count_position, font, 1.2, (0, 0, 0), 2, cv2.LINE_AA)

                                    # 스쿼트 감지
                                    if exercise_type.lower() == 'squat':
                                        # 준비자세
                                        right_squat = 140 <= right_hip_knee_ankle_angle <= 200
                                        left_squat = 140 <= left_hip_knee_ankle_angle <= 200

                                        # 준비자세가 되면
                                        if right_squat and left_squat:
                                            if not detected:
                                                count_s += 1
                                                # detected 완벽한 자세 감지
                                                detected = True
                                                if count_s > 0:
                                                    print(f"운동 횟수: {count_s}회")
                                                    count = count_s
                                                    exercise_queue.put(count)
                                                   
                                            if less_count:
                                                print('덜 앉음')
                                                text_to_speech(f'더 앉아요', lang='ko', filename='outSound.wav')
                                                less_count = False

                                            if more_count:
                                                print('깊게 앉음')
                                                text_to_speech(f'깊게 앉았어요', lang='ko', filename='outSound.wav')
                                                more_count = False
 
                                       # 덜앉음
                                        if (220 <= right_hip_knee_ankle_angle < 250 and 220 <= left_hip_knee_ankle_angle < 250) or \
                            (100 <= right_hip_knee_ankle_angle < 140 and 100 <= left_hip_knee_ankle_angle < 140) and not (
                            right_squat and left_squat):
                                            if detected and not more_count:
                                                less_count = True

                                       # 앉았을 때
                                        if (80 <= right_hip_knee_ankle_angle <= 100 or 80 <= left_hip_knee_ankle_angle <= 100) or \
                            (250 <= right_hip_knee_ankle_angle <= 270 or 250 <= left_hip_knee_ankle_angle <= 270) and not (
                        right_squat and left_squat):
                                            if not more_count:
                                                detected = False
                                                less_count = False

                                       # 깊게 앉음
                                        if (0 <= right_knee_hip_shoulder_angle <= 50 or 0 <= left_knee_hip_shoulder_angle <= 50) or \
                        (300 <= right_knee_hip_shoulder_angle <= 350 or 300 <= left_knee_hip_shoulder_angle <= 350):
                                            more_count = True
                                            detected = True
                                            less_count = False

                                  # 푸쉬업 감지
                                    if exercise_type.lower() == 'pushup':
                                        right_pushup = 150 <= right_shoulder_elbow_wrist_angle <= 210
                                        left_pushup = 150 <= left_shoulder_elbow_wrist_angle <= 210
                                        pushup_hip = 160 <= right_ankle_hip_shoulder_angle <= 200 and 160 <= left_ankle_hip_shoulder_angle <= 200

                                        if right_pushup and left_pushup and pushup_hip:
                                            if not detected:
                                                count_s += 1
                                                detected = True
                                                if count_s > 0:
                                                    print(f"운동 횟수: {count_s}회")
                                                    count = count_s
                                                    exercise_queue.put(count)
                                                   
                                            if less_count:
                                                print('팔 덜 내림')
                                                text_to_speech(f'더 내려가요', lang='ko', filename='outSound.wav')
                                                less_count = False
                                            if more_count:
                                                more_count = False

                                    # 팔 덜굽힐 때
                                        if (100 <= right_shoulder_elbow_wrist_angle <= 140 and 100 <= left_shoulder_elbow_wrist_angle <= 140) or \
                        (220 <= right_shoulder_elbow_wrist_angle <= 260 and 220 <= left_shoulder_elbow_wrist_angle <= 260) and not \
                        (right_pushup and left_pushup):
                                            if detected:
                                                less_count = True

                    # 팔이 벌어지고 다시 펴졌을 때
                                        if (40 <= right_shoulder_elbow_wrist_angle <= 80 or 40 <= left_shoulder_elbow_wrist_angle <= 80) or (270<= right_shoulder_elbow_wrist_angle <= 330 or 270 <= left_shoulder_elbow_wrist_angle <= 330) and not (right_pushup or left_pushup) :
                                            if not more_count :
                                                detected = False
                                                less_count = False

                                        # 엉덩이 높이 벗어날 때
                                        if not pushup_hip :
                                            if not more_count :
                                                detected = True
                                                more_count = True
                                                print('엉덩이 벗어남')
                                                text_to_speech(f'엉덩이 벗어났어요', lang='ko', filename='outSound.wav')

                                        # 풀업 감지
                                    if exercise_type.lower() == 'pullup':
                                        # 올라갈 때
                                        pullup = (0 <= right_shoulder_elbow_wrist_angle <= 50 or 300 <= right_shoulder_elbow_wrist_angle <= 350) and \
                            (0 <= left_shoulder_elbow_wrist_angle <= 50 or 300 <= left_shoulder_elbow_wrist_angle <= 350)

                                        if pullup :
                                            if not detected :
                                                count += 1
                                                detected = True
                                                print(f"운동 횟수: {count}회")
                                                exercise_queue.put(count)
                                                more_count = False
                                               
                                            if less_count :
                                                print('덜 내려감')
                                                text_to_speech(f'더 내려가요', lang='ko', filename='outSound.wav')
                                                less_count = False


                                        if (250 <= right_shoulder_elbow_wrist_angle <= 300 or 50 <= right_shoulder_elbow_wrist_angle <= 100) and \
                    (250 <= left_shoulder_elbow_wrist_angle <= 300 or 50 <= left_shoulder_elbow_wrist_angle <= 100) :
                                                # 덜 올라갈 때
                                            if not detected :
                                                more_count = True
                                                # 덜 내려갈 때
                                            if detected :
                                                less_count = True

                                                # 팔 다 폈을 때
                                        if (200 <= right_shoulder_elbow_wrist_angle <= 250 or 100 <= right_shoulder_elbow_wrist_angle <= 150) and \
                    (200 <= left_shoulder_elbow_wrist_angle <= 250 or 100 <= left_shoulder_elbow_wrist_angle <= 150) :
                                            detected = False
                                            less_count = False
                                            if more_count :
                                                print('덜 올라감')
                                                text_to_speech(f'더 올라가요', lang='ko', filename='outSound.wav')
                                                more_count = False



                                    # 관절을 선으로 이어서 그리기
                                    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

         
                               
                                    # 프레임을 녹화 동영상에 저장
                                    out.write(frame)
                   

                                    if count >= input_count :
                                        print(f"운동 횟수 {input_count} 달성 또는 시간 초과! 동영상 녹화를 중지합니다.")
                                        input_video_path = exercise_file_path
                                        output_video_path = exercise_file_path.split(".")[0] + "_converted.mp4"
                                        out.release()
                                        os.system(f'ffmpeg -i {input_video_path} -c:v libx264 -crf 23 -c:a aac -strict -2 {output_video_path}')
         
   
                                        # 데이터베이스의 'exercise_data' 노드에 대한 참조 생성
                                        ref = db.reference("exercise_data")
                                        if exercise.lower() == 'pullup':
                                            update_exercise_data(exercise_type=exercise,pullup_count=count, path=output_video_path.split("/")[-1])
                                        if exercise.lower() == 'pushup':
                                            update_exercise_data(exercise_type=exercise,pushup_count=count, path=output_video_path.split("/")[-1])
                                        if exercise.lower() == 'squat':
                                            update_exercise_data(exercise_type=exercise,squat_count=count, path=output_video_path.split("/")[-1])
                                        # 휴식 시간만큼 대기
                                        if set_count < int(input_sets) - 1:
                                            mp3_file_path = "/home/altong/Desktop/project/bgm.mp3"
                                            if time.time() - last_count_time < int(input_rest):
                                                print(f"\n휴식 시간 {input_rest}초를 기다립니다...\n")
                                                text_to_speech(f'{input_rest}초동안 휴식시간이에요', lang='ko', filename='outSound.wav')
                                                time.sleep(int(input_rest))
                                                #pygame.init()
                                                #pygame.mixer.init()
                                                #play_audio(mp3_file_path)
                                                #last_count_time = time.time()
                                                #while time.time() - start_time < int(input_rest):
                                                #    pygame.time.Clock().tick(int(input_rest))
                                                #    pygame.mixer.music.stop()
                                            else:
                                                break

                                        cap.release()
                                        cv2.destroyAllWindows()      
                                       
                   
                            # 화면에 출력
                                cv2.imshow('Pose Detection', frame)


                                if cv2.waitKey(1) & 0xFF == ord('q'):
                                    break
       

                        user_state = False
                           
                       

def consumer_thread():
    global exercise_queue

    while True:
        # Check if the queue is not empty and retrieve counts
        if not exercise_queue.empty():
            retrieved_count = exercise_queue.get()
            text_to_speech(f'{retrieved_count}회', lang='ko', filename='outSound.wav')
            #pygame.init()
            #pygame.mixer.init()

# Main loop
# if user_state :

    # Start the producer and consumer threads
producer = threading.Thread(target=producer_thread)
consumer = threading.Thread(target=consumer_thread)

producer.start()
consumer.start()

# Wait for the producer to finish
producer.join()

# You can add logic to gracefully stop the consumer thread if needed
consumer.join()  